## Scraping de Noticias y Serialización en JSONL


- URL: https://www.reuters.com/world/

   - Extraer al menos 20 noticias recientes con los siguientes campos:
     - id (un identificador único, puedes generar un hash o número incremental)
     - titulo
     - fecha (en formato ISO si está disponible)
     - url
     - fuente (nombre del sitio)
     - autor
     - capturado_ts (timestamp de la captura en UTC)
     - resumen (campo adicional de interés)

3. **Serialización en JSONL:**
   - Guardar cada noticia como un objeto JSON en una línea dentro del archivo `data/raw/noticias.jsonl`.


In [15]:
import requests
from bs4 import BeautifulSoup
import json
import os
import hashlib
from datetime import datetime, timezone
import re

In [ ]:
def scrape_bbc_mundo_tecnologia(url="https://www.bbc.com/mundo/topics/cyx5krnw38vt", num_news=20):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except Exception as e:
        print(f"Error en la petición HTTP: {e}")
        return []
    soup = BeautifulSoup(response.content, 'html.parser')
    noticias = []
    news_items = soup.find_all('li', class_='bbc-t44f9r')
    print(f"Encontrados {len(news_items)} posibles elementos de noticias")
    for i, item in enumerate(news_items):
        if len(noticias) >= num_news:
            break
        try:
            # Título y URL
            h2 = item.find('h2')
            a = h2.find('a') if h2 else None
            if not a:
                continue
            titulo = a.get_text().strip()
            noticia_url = a['href']
            if not noticia_url.startswith('http'):
                noticia_url = f"https://www.bbc.com{noticia_url}"
            # Fecha
            fecha_elem = item.find('time')
            fecha = fecha_elem.get('datetime') if fecha_elem and fecha_elem.has_attr('datetime') else (fecha_elem.get_text().strip() if fecha_elem else "Fecha no disponible")
            # Resumen (opcional, puede ser el alt de la imagen o el texto del h2)
            img = item.find('img')
            resumen = img['alt'].strip() if img and img.has_attr('alt') else titulo
            # Autor: requiere entrar a la noticia
            autor = "Autor no disponible"
            try:
                art_resp = requests.get(noticia_url, headers=headers, timeout=10)
                art_resp.raise_for_status()
                art_soup = BeautifulSoup(art_resp.content, 'html.parser')
                byline_section = art_soup.find('section', {'role': 'region', 'aria-labelledby': 'article-byline'})
                if byline_section:
                    autor_span = byline_section.find('span', class_='bbc-of1z6f')
                    if autor_span:
                        autor = autor_span.get_text().strip()
            except Exception as e:
                pass
            # ID único
            # Extraer el id de la noticia desde el URL (lo que sigue a 'articles/')
            noticia_id = "id_no_encontrado"
            match = re.search(r'/articles/([^/?#]+)', noticia_url)
            if match:
                noticia_id = match.group(1)
            # Timestamp de captura
            capturado_ts = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
            noticia = {
                "id": noticia_id,
                "titulo": titulo,
                "fecha": fecha,
                "url": noticia_url,
                "fuente": "BBC Mundo Tecnología",
                "autor": autor,
                "capturado_ts": capturado_ts,
                "resumen": resumen[:200] + "..." if len(resumen) > 200 else resumen
            }
            noticias.append(noticia)
        except Exception as e:
            print(f"Error procesando noticia {i}: {e}")
            continue
    return noticias

output_path = "../../data/raw/noticias.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

noticias = scrape_bbc_mundo_tecnologia(num_news=20)

with open(output_path, "w", encoding="utf-8") as f:
    for noticia in noticias:
        f.write(json.dumps(noticia, ensure_ascii=False) + "\n")

print(f"Guardado {len(noticias)} noticias en {output_path}")


Encontrados 24 posibles elementos de noticias
Guardado 20 noticias en ../../data/raw/noticias.jsonl


In [20]:
# Mostrar las primeras 3 líneas del archivo JSONL generado
with open(output_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 2:
            break


{"id": "c4gv9r1120mo", "titulo": "Los cerdos y los insectos que están ayudando a encontrar a los desaparecidos en México", "fecha": "2025-10-01", "url": "https://www.bbc.com/mundo/articles/c4gv9r1120mo", "fuente": "BBC Mundo Tecnología", "autor": "Alejandro Millán Valencia", "capturado_ts": "2025-10-03T04:51:32Z", "resumen": "Científicos preparan una fosa para poner el cuerpo de un animal para ser analizados en su proceso de descomposición."}
{"id": "cz0827j1nvdo", "titulo": "Las mujeres de Afganistán pierden su \"última esperanza\" tras el bloqueo de internet por parte del Talibán", "fecha": "2025-10-01", "url": "https://www.bbc.com/mundo/articles/cz0827j1nvdo", "fuente": "BBC Mundo Tecnología", "autor": "Mahfouz Zubaide", "capturado_ts": "2025-10-03T04:51:32Z", "resumen": "Mujer afgana"}
{"id": "ckgqry9x72ko", "titulo": "\"Nunca más tendrás que volver a trabajar\": cómo un grupo de delincuentes intentó sobornarme para hackear la BBC", "fecha": "2025-09-30", "url": "https://www.bbc.co

In [21]:
import pandas as pd
import json
from collections import Counter
import re

with open('../../data/raw/noticias.jsonl', 'r', encoding='utf-8') as f:
    records = [json.loads(line) for line in f if line.strip()]

df = pd.DataFrame(records)

# 1. Número total de registros
num_registros = len(df)
print(f"Número total de registros: {num_registros}")

# 2. Porcentaje de valores nulos por campo
porcentaje_nulos = df.isnull().mean() * 100
print("\nPorcentaje de valores nulos por campo:")
print(porcentaje_nulos)

# 3. Duplicados por id y por url
duplicados_id = df.duplicated(subset=['id']).sum()
duplicados_url = df.duplicated(subset=['url']).sum()
print(f"\nDuplicados por id: {duplicados_id}")
print(f"Duplicados por url: {duplicados_url}")

# 4. Consistencia en el formato de fechas y URLs
# Fecha: formato esperado YYYY-MM-DD
fecha_regex = re.compile(r'^\d{4}-\d{2}-\d{2}$')
fechas_validas = df['fecha'].apply(lambda x: bool(fecha_regex.match(str(x))))
print(f"\nFechas con formato válido (YYYY-MM-DD): {fechas_validas.sum()} / {num_registros}")

# URL: debe empezar con http(s)://
url_regex = re.compile(r'^https?://')
urls_validas = df['url'].apply(lambda x: bool(url_regex.match(str(x))))
print(f"URLs con formato válido: {urls_validas.sum()} / {num_registros}")

# Mostrar resumen
print("\nResumen de calidad de datos:")
display(df.head())


Número total de registros: 20

Porcentaje de valores nulos por campo:
id              0.0
titulo          0.0
fecha           0.0
url             0.0
fuente          0.0
autor           0.0
capturado_ts    0.0
resumen         0.0
dtype: float64

Duplicados por id: 0
Duplicados por url: 0

Fechas con formato válido (YYYY-MM-DD): 20 / 20
URLs con formato válido: 20 / 20

Resumen de calidad de datos:


,id,titulo,fecha,url,fuente,autor,capturado_ts,resumen
0,c4gv9r1120mo,Los cerdos y los insectos que están ayudando a...,2025-10-01,https://www.bbc.com/mundo/articles/c4gv9r1120mo,BBC Mundo Tecnología,Alejandro Millán Valencia,2025-10-03T04:51:32Z,Científicos preparan una fosa para poner el cu...
1,cz0827j1nvdo,"Las mujeres de Afganistán pierden su ""última e...",2025-10-01,https://www.bbc.com/mundo/articles/cz0827j1nvdo,BBC Mundo Tecnología,Mahfouz Zubaide,2025-10-03T04:51:32Z,Mujer afgana
2,ckgqry9x72ko,"""Nunca más tendrás que volver a trabajar"": cóm...",2025-09-30,https://www.bbc.com/mundo/articles/ckgqry9x72ko,BBC Mundo Tecnología,Joe Tidy,2025-10-03T04:51:33Z,Joe Tidy looking down at a phone. He has short...
3,c20z3gnd9l0o,El Talibán corta internet y causa un apagón de...,2025-09-30,https://www.bbc.com/mundo/articles/c20z3gnd9l0o,BBC Mundo Tecnología,Anbarasan Ethirajan,2025-10-03T04:51:33Z,Hombre en Afganistán revisa su teléfono.
4,c4gk21318dxo,"La familia Ellison, la nueva dinastía de los m...",2025-09-29,https://www.bbc.com/mundo/articles/c4gk21318dxo,BBC Mundo Tecnología,Natalie Sherman,2025-10-03T04:51:34Z,Larry Ellison (izq.) junto a su hija Megan y s...
